## urllib 演示

### 测试urlopen：向百度发送请求，并得到响应

In [1]:
import urllib.request as request

In [2]:
# 得到：响应对象
response = request.urlopen('http://www.baidu.com/')

# 获取响应内容,html源码
html = response.read().decode()

# 打印响应内容
# print(html)

print(response.geturl())

print(response.getcode())

http://www.baidu.com/
200


### 非人的请求user-agent:请求测试网站

In [3]:
# User-Agent 为python
res_1 = request.urlopen('http://httpbin.org/get')
print(res_1.read().decode())

{
  "args": {}, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Host": "httpbin.org", 
    "User-Agent": "Python-urllib/3.6"
  }, 
  "origin": "43.254.90.134, 43.254.90.134", 
  "url": "https://httpbin.org/get"
}



### 伪造人为的请求 修改user-agent头字段

In [4]:
url = 'http://httpbin.org/get'

# 1,创造请求对象
req = request.Request(
---第33页正在抓取---
---第33页抓取成功
---第34页正在抓取---
---第34页抓取成功
---第35页正在抓取---
---第35页抓取成功
---第36页正在抓取---
    url=url,
    headers={'User-Agent':'Mozilla/5.0'}
)

# 2,发请求获取响应对象
res = request.urlopen(req)

# 3,提取相应对象内容
print(res.read().decode())

{
  "args": {}, 
  "headers": {
    "Accept-Encoding": "identity", 
    "Host": "httpbin.org", 
    "User-Agent": "Mozilla/5.0"
  }, 
  "origin": "43.254.90.134, 43.254.90.134", 
  "url": "https://httpbin.org/get"
}



### url 地址编码模块urllib.parse 演示：

In [8]:
import urllib

In [22]:
# 将搜索的内容保存到本地文件

# 拼url地址 urlencode: 将查询字典转换为编码后的请求字符串
url = 'https://baidu.com/s?{}'
params = {
    'wd':'赵丽颖',
}
params = urllib.parse.urlencode(params)
full_url = url.format(params)

# 1.创建请求对象 Request
req = urllib.request.Request(
    url=full_url,
    headers={
        'User-Agent':'Mozilla/5.0'
    }
)

# 2.获取响应对象 urlopen
res = urllib.request.urlopen(req)

# 3.获取响应内容 read()
html = res.read().decode()
with open('赵丽颖.html','w',encoding='utf8') as f_write:
    f_write.write(html)

In [29]:
# 使用quote拼接
url = 'http://www.baidu.com/s?wd={}&pn={}'
params = (urllib.parse.quote('赵丽颖'),10)
full_url = url.format(params[0],params[1])

# 人为修改请求对象
req = urllib.request.Request(
    url=full_url,
    headers={
        'User-Agent':'Morilla/5.0'
    }
)

# 获取响应对象
res = urllib.request.urlopen(req)

# 读取数据到文件
html = res.read().decode()
with open('赵丽颖.html','w') as f:
    f.write(html)

In [32]:
# quote 和 unquote
string = '迪丽热巴'
string_quote = urllib.parse.quote(string)
print(string_quote)
string = urllib.parse.unquote(string_quote)
print(string)

%E8%BF%AA%E4%B8%BD%E7%83%AD%E5%B7%B4
迪丽热巴


### 抓取百度贴吧50页的数据 并对比单线程 多进程 多线程的效率

### 效率对比： 单线程：50s    多线程：10s    多进程：8s

In [1]:
# 使用单线程爬取50页
import time
# 首先确认是否是静态页面不是动态js请求的
start = time.time()
url = 'http://tieba.baidu.com/f?kw={}&pn={}'
name = input('请输入你要爬取的贴吧名称：')
name_quote = urllib.parse.quote(name)

url_list = []
for i in range(50):
    num = i * 50
    full_url = url.format(name_quote,str(num))
    url_list.append(full_url)
    
# 人为仿造请求对象并返回数据保存  使用单线程实现
for i,true_url in enumerate(url_list):
    print('---第{}页正在抓取---'.format(i+1))
    req = urllib.request.Request(
        url=true_url,
        headers={
            'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1'
        }
    )
    res = urllib.request.urlopen(req)
    html = res.read().decode()
    with open('{}_{}.html'.format(name,i+1),'w') as f:
        f.write(html)
    print('---第{}页抓取成功---'.format(i+1))

end = time.time()
print('执行时间为:%.2f' % (end-start))

KeyboardInterrupt: 

In [44]:
# 使用多线程爬取50页
import time
import threading
# 首先确认是否是静态页面不是动态js请求的
start = time.time()
url = 'http://tieba.baidu.com/f?kw={}&pn={}'
name = input('请输入你要爬取的贴吧名称：')
name_quote = urllib.parse.quote(name)

url_list = []
for i in range(50):
    num = i * 50
    full_url = url.format(name_quote,str(num))
    url_list.append(full_url)
    
# 编写线程的target函数
def tieba_spider(url,name,num):
    req = urllib.request.Request(
        url=url,
        headers={
            'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1'
        }
    )
    res = urllib.request.urlopen(req)
    html = res.read().decode()
    with open('{}_{}.html'.format(name,num),'w') as f:
        f.write(html)
    print('---第{}页抓取成功---'.format(num))
    

# 人为仿造请求对象并返回数据保存  使用多线程实现
t_list = []
for i,true_url in enumerate(url_list):
    print('---第{}页正在抓取---'.format(i+1))
    t = threading.Thread(target=tieba_spider, args=(true_url,name,i+1))
    t_list.append(t)
    t.start()
    

for thread in t_list:
    thread.join()

end = time.time()
print('执行时间为:%.2f' % (end-start))

请输入你要爬取的贴吧名称：迪丽热巴
---第1页正在抓取---
---第2页正在抓取---
---第3页正在抓取---
---第4页正在抓取---
---第5页正在抓取---
---第6页正在抓取---
---第7页正在抓取---
---第8页正在抓取---
---第9页正在抓取---
---第10页正在抓取---
---第11页正在抓取---
---第12页正在抓取---
---第13页正在抓取---
---第14页正在抓取---
---第15页正在抓取---
---第16页正在抓取---
---第17页正在抓取---
---第18页正在抓取---
---第19页正在抓取---
---第20页正在抓取---
---第21页正在抓取---
---第22页正在抓取---
---第23页正在抓取---
---第24页正在抓取---
---第25页正在抓取---
---第26页正在抓取---
---第27页正在抓取---
---第28页正在抓取---
---第29页正在抓取---
---第30页正在抓取---
---第31页正在抓取---
---第32页正在抓取---
---第33页正在抓取---
---第34页正在抓取---
---第35页正在抓取---
---第36页正在抓取---
---第37页正在抓取---
---第38页正在抓取---
---第39页正在抓取---
---第40页正在抓取---
---第41页正在抓取---
---第42页正在抓取---
---第43页正在抓取---
---第44页正在抓取---
---第45页正在抓取---
---第46页正在抓取---
---第47页正在抓取---
---第48页正在抓取---
---第49页正在抓取---
---第50页正在抓取---
---第47页抓取成功---
---第30页抓取成功---
---第17页抓取成功---
---第25页抓取成功---
---第40页抓取成功---
---第45页抓取成功---
---第41页抓取成功------第27页抓取成功---

---第10页抓取成功---
---第34页抓取成功---
---第11页抓取成功---
---第4页抓取成功---
---第3页抓取成功---
---第15页抓取成功---
---第5页抓取成功---
---第22页抓取成功---
---第

In [42]:
import time
import multiprocessing
# 首先确认是否是静态页面不是动态js请求的
start = time.time()
url = 'http://tieba.baidu.com/f?kw={}&pn={}'
name = input('请输入你要爬取的贴吧名称：')
name_quote = urllib.parse.quote(name)

url_list = []
for i in range(50):
    num = i * 50
    full_url = url.format(name_quote,str(num))
    url_list.append(full_url)
    
# 编写线程的target函数
def tieba_spider(url,name,num):
    req = urllib.request.Request(
        url=url,
        headers={
            'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1'
        }
    )
    res = urllib.request.urlopen(req)
    html = res.read().decode()
    with open('{}_{}.html'.format(name,num),'w') as f:
        f.write(html)
    print('---第{}页抓取成功---'.format(num))
    

# 人为仿造请求对象并返回数据保存  使用多线程实现
p_list = []
for i,true_url in enumerate(url_list):
    print('---第{}页正在抓取---'.format(i+1))
    p = multiprocessing.Process(target=tieba_spider, args=(true_url,name,i+1))
    p_list.append(p)
    p.start()
    

for process in p_list:
    process.join()

end = time.time()
print('执行时间为:%.2f' % (end-start))

请输入你要爬取的贴吧名称：迪丽热巴
---第1页正在抓取---
---第2页正在抓取---
---第3页正在抓取---
---第4页正在抓取---
---第5页正在抓取---
---第6页正在抓取---
---第7页正在抓取---
---第8页正在抓取---
---第9页正在抓取---
---第10页正在抓取---
---第11页正在抓取---
---第12页正在抓取---
---第13页正在抓取---
---第14页正在抓取---
---第15页正在抓取---
---第16页正在抓取---
---第17页正在抓取---
---第18页正在抓取---
---第19页正在抓取---
---第20页正在抓取---
---第21页正在抓取---
---第22页正在抓取---
---第23页正在抓取---
---第24页正在抓取---
---第25页正在抓取---
---第26页正在抓取---
---第27页正在抓取---
---第28页正在抓取---
---第29页正在抓取---
---第30页正在抓取---
---第31页正在抓取---
---第32页正在抓取---
---第33页正在抓取---
---第34页正在抓取---
---第35页正在抓取---
---第36页正在抓取---
---第37页正在抓取---
---第38页正在抓取---
---第39页正在抓取---
---第40页正在抓取---
---第41页正在抓取---
---第42页正在抓取---
---第43页正在抓取---
---第44页正在抓取---
---第45页正在抓取---
---第46页正在抓取---
---第47页正在抓取---
---第48页正在抓取---
---第49页正在抓取---
---第50页正在抓取---
---第24页抓取成功---
---第1页抓取成功---
---第3页抓取成功---
---第43页抓取成功---
---第49页抓取成功---
---第9页抓取成功---
---第27页抓取成功---
---第5页抓取成功---
---第47页抓取成功---
---第28页抓取成功---
---第40页抓取成功---
---第23页抓取成功---
---第46页抓取成功---
---第25页抓取成功---
---第30页抓取成功---
---第50页抓取成功---
---第2

### 正则模块 re 爬虫专用的非贪婪模式的使用

In [53]:
import re

# re.S 用来让 '.' 占位符匹配\n  不指定则不匹配
# pattern = re.compile(r'',re.S)

# 贪婪匹配
html = '''
<div><p>九霄龙吟惊天变</p></div>
<div><p>风云际会浅水游</p></div>
'''
pattern = re.compile(r'<div><p>(.*)</p></div>', re.S)
result = pattern.findall(html)
print(result)

# 非贪婪匹配 -> 爬虫专用
pattern = re.compile(r'<div><p>(.*?)</p></div>', re.S)
result = pattern.findall(html)
print(result)

['九霄龙吟惊天变</p></div>\n<div><p>风云际会浅水游']
['九霄龙吟惊天变', '风云际会浅水游']


In [59]:
import re

s = 'A B C D'
p1 = re.compile('\w+\s+\w+')
print(p1.findall(s))

p2 = re.compile('(\w+)\s+\w+')
print(p2.findall(s))

p3 = re.compile('(\w+)\s+(\w+)')
print(p3.findall(s))

['A B', 'C D']
['A', 'C']
[('A', 'B'), ('C', 'D')]


In [67]:
html = """<div class="animal">
<div class="animal">
    <p class="name">
		<a title="Tiger"></a>
    </p>
    <p class="content">
		Two tigers two tigers run fast
    </p>
</div>

<div class="animal">
    <p class="name">
		<a title="Rabbit"></a>
    </p>

    <p class="content">
		Small white rabbit white and white
    </p>
</div>"""

pattern = re.compile(r'.*?title="(.*?)".*?content">(.*?)</p>', re.S)
result = pattern.findall(html)
result = [(tup[0],tup[1].strip()) for tup in result]
for animal,detail in result:
    print('动物名称:',animal)
    print('动物描述:',detail)

动物名称: Tiger
动物描述: Two tigers two tigers run fast
动物名称: Rabbit
动物描述: Small white rabbit white and white


### 使用多进程爬取猫眼电影-榜单-top100榜的电影名称、主演、上演时间

#### 第一步：获取前10个页面

In [2]:
import multiprocessing
import random
import time
import urllib

start = time.time()
# 随机user-agent 列表
user_agent_list = [
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0) Gecko/20100101 Firefox/6.0',
    'Opera/9.80 (Windows NT 6.1; U; zh-cn) Presto/2.9.168 Version/11.50',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; GTB7.0'
]

# 获取所有的url列表
url = 'https://maoyan.com/board/4?offset={}'
url_list = []
for num in range(10):
    offset = num * 10
    full_url = url.format(offset)
    url_list.append(full_url)
    
# 进程的target函数
def spider(url,num):
    req = urllib.request.Request(
        url=url,
        headers={
            'User-Agent':random.choice(user_agent_list)
        }
    )
    res = urllib.request.urlopen(req)
    html = res.read().decode()
    with open('猫眼电影_{}'.format(num), 'w') as f:
        f.write(html)
    print('第{}页爬取成功'.format(num))

# 多进程列表
p_list = []
for i in range(10):
    print('第{}页正在爬取'.format(i+1))
    time.sleep(random.random())
    p = multiprocessing.Process(target=spider,args=(url_list[i],i+1))
    p_list.append(p)
    p.start()
    
# 释放进程资源
for process in p_list:
    process.join()
    
end = time.time()
print('耗费时间:%.2f'%(end-start))

第1页正在爬取
第1页爬取成功
第2页正在爬取
第2页爬取成功
第3页正在爬取
第3页爬取成功
第4页正在爬取
第4页爬取成功
第5页正在爬取
第5页爬取成功
第6页正在爬取
第6页爬取成功
第7页正在爬取
第7页爬取成功
第8页正在爬取
第9页正在爬取
第8页爬取成功
第9页爬取成功
第10页正在爬取
第10页爬取成功
耗费时间:5.34


#### 第二步：从文件中匹配需要的内容：电影名称、主演、上映时间

In [95]:
# 编写pattern匹配数据
pattern_movie = re.compile(r'.*?title="(.*?)".*?"star">(.*?)</p>.*?(上映时间：.*?)</p>', re.S)
movie_detail = []
for i in range(1,11):
    with open('猫眼电影_{}'.format(i),'r') as f_read:
        string = f_read.read()
        movie_detail.extend(pattern_movie.findall(string))

# 清洗数据
movie_detail = [ (('名称:'+tup[0]),tup[1].strip(),tup[2]) for tup in movie_detail]
movie_detail = ['    '.join(_) for _ in movie_detail]

# 将列表数据写入到文件中
with open('cat`s_eye_movie_detail','w') as f:
    for i,movie in enumerate(movie_detail):
        string = '第{}名'.format(i+1).center(10) + movie + '\n'
        f.write(string)

#### 第三步：将内容存储在csv文件中

In [31]:
import re
import csv


# 编写pattern匹配数据
pattern_movie = re.compile(r'.*?title="(.*?)".*?"star">(.*?)</p>.*?(上映时间：.*?)</p>', re.S)
movie_detail = []
for i in range(1,11):
    with open('猫眼电影_{}'.format(i),'r') as f_read:
        string = f_read.read()
        movie_detail.extend(pattern_movie.findall(string))

# 数据清洗
movie_detail = [(tup[0].strip(), tup[1].strip()[3:], tup[2].strip()[5:15]) for tup in movie_detail]

# 将数据存储为csv文件
with open('cat`s_eye_movie_detail.csv','w') as f:
    writer = csv.writer(f)
    writer.writerow(['电影名称','主演','上映时间'])
    writer.writerows(movie_detail)
#     for row in movie_detail:
#         writer.writerow(row)
print(movie_detail)


[('霸王别姬', '张国荣,张丰毅,巩俐', '1993-01-01'), ('肖申克的救赎', '蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', '1994-09-10'), ('罗马假日', '格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', '1953-09-02'), ('这个杀手不太冷', '让·雷诺,加里·奥德曼,娜塔莉·波特曼', '1994-09-14'), ('泰坦尼克号', '莱昂纳多·迪卡普里奥,凯特·温丝莱特,比利·赞恩', '1998-04-03'), ('唐伯虎点秋香', '周星驰,巩俐,郑佩佩', '1993-07-01'), ('魂断蓝桥', '费雯·丽,罗伯特·泰勒,露塞尔·沃特森', '1940-05-17'), ('乱世佳人', '费雯·丽,克拉克·盖博,奥利维娅·德哈维兰', '1939-12-15'), ('天空之城', '寺田农,鹫尾真知子,龟山助清', '1992-05-01'), ('辛德勒的名单', '连姆·尼森,拉尔夫·费因斯,本·金斯利', '1993-12-15'), ('喜剧之王', '周星驰,莫文蔚,张柏芝', '1999-02-13'), ('音乐之声', '朱莉·安德鲁斯,克里斯托弗·普卢默,埃琳诺·帕克', '1965-03-02'), ('大闹天宫', '邱岳峰,毕克,富润生', '1965-12-31'), ('春光乍泄', '张国荣,梁朝伟,张震', '1997-05-30'), ('剪刀手爱德华', '约翰尼·德普,薇诺娜·瑞德,黛安·韦斯特', '1990-12-06'), ('海上钢琴师', '蒂姆·罗斯,普路特·泰勒·文斯,比尔·努恩', '1998-10-28'), ('美丽人生', '罗伯托·贝尼尼,尼可莱塔·布拉斯基,乔治·坎塔里尼', '1997-12-20'), ('黑客帝国', '基努·里维斯,凯瑞-安·莫斯,劳伦斯·菲什伯恩', '2000-01-14'), ('哈利·波特与魔法石', '丹尼尔·雷德克里夫,鲁伯特·格林特,艾玛·沃特森', '2002-01-26'), ('指环王3：王者无敌', '伊莱贾·伍德,伊恩·麦克莱恩,丽芙·泰勒', '2004-03-15'), ('蝙蝠侠：黑暗骑士', '克里斯蒂安·贝尔,希斯·莱杰,阿伦·伊克哈特', '2008-07-18')

#### 第四步：将csv文件存储到数据库中

In [28]:
import pymysql
import time

# 连接mysql，并编写sql语句
db = pymysql.connect('localhost','root','123456','spider',use_unicode=True,charset='utf8')
insert_sql = 'insert into movie(name,actor,show_time) values(%s,%s,%s)'
cursor = db.cursor()

# csv读取文件到列表
movie_list = []
with open('cat`s_eye_movie_detail.csv','r') as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        movie_list.append(row)

# 将列表数据保存到数据库表movie中
# 使用execute_many 对比 execute 效率
try:
    
    %time cursor.executemany(insert_sql,movie_list)
except Exception as e:
    print(e)
    db.rollback()
    
# 提交数据并删除指针        
db.commit()
cursor.close()
db.close()

CPU times: user 3.42 ms, sys: 4.15 ms, total: 7.57 ms
Wall time: 18.2 ms


#### 第四步：存储数据到mongodb

In [32]:
import pymongo

# 连接对象
con = pymongo.MongoClient('localhost',27017)

# 库对象 不存在自动创建
db = con['spider']

# 集合对象
my_set = db['maoyan']

for movie in movie_detail:
    my_set.insert_one({'name':movie[0],'actors':movie[1],'time':movie[2]})